In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import ast
import re
import sys
print('sys.executable =', sys.executable)
import importlib
print('keybert spec =', importlib.util.find_spec('keybert'))
import seaborn as sns
from keybert import KeyBERT
from nltk.corpus import stopwords

# Load dataset
df = pd.read_csv("Uitgebreide_VKM_dataset.csv")

sys.executable = c:\Users\Dhr. Kootwijk\AppData\Local\Programs\Python\Python310\python.exe
keybert spec = ModuleSpec(name='keybert', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000002311D9982E0>, origin='c:\\Users\\Dhr. Kootwijk\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\keybert\\__init__.py', submodule_search_locations=['c:\\Users\\Dhr. Kootwijk\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\keybert'])


c:\Users\Dhr. Kootwijk\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df.columns = df.columns.str.strip()
print(df.columns)

Index(['id', 'name', 'shortdescription', 'description', 'content',
       'studycredit', 'location', 'contact_id', 'level', 'learningoutcomes',
       'Rood', 'Groen', 'Blauw', 'Geel', 'module_tags',
       'interests_match_score', 'popularity_score', 'estimated_difficulty',
       'available_spots', 'start_date'],
      dtype='object')


In [ ]:
# Basislijst Nederlandse stopwoorden (kan je uitbreiden)
DUTCH_STOPWORDS = {
    'de', 'het', 'een', 'en', 'of', 'in', 'op', 'aan', 'met', 'voor', 'van',
    'naar', 'bij', 'over', 'onder', 'boven', 'tussen', 'tot', 'door', 'tegen',
    'uit', 'te', 'ter', 'als', 'dat', 'die', 'dit', 'dan', 'maar', 'ook',
    'nog', 'al', 'eens', 'er', 'hier', 'daar', 'waar', 'wie', 'wat', 'wel',
    'niet', 'geen', 'bijvoorbeeld', 'zoals', 'zijn', 'is', 'was', 'wordt',
    'worden', 'heb', 'hebt', 'heeft', 'hebben', 'had', 'hadden'
}

def remove_color_columns(df, color_cols=None):
    """Verwijder de kolommen met kleuren als ze bestaan."""
    if color_cols is None:
        color_cols = ['Rood', 'Geel', 'Blauw', 'Groen']

    cols_to_drop = [col for col in color_cols if col in df.columns]
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
    return df

def remove_rows_by_id(df, ids_to_remove):
    """Verwijder rijen op basis van id-lijst."""
    return df[~df['id'].isin(ids_to_remove)]

def _update_short_description_row(row):
    """Row-helper voor shortdescription."""
    desc = str(row.get('description', '')).strip()
    cont = str(row.get('content', '')).strip()
    short = str(row.get('shortdescription', '')).strip()

    # Only change when short_description == 'Ntb' of 'nan'
    if short.lower() == 'ntb' or short.lower() == 'nan':
        # If description or content is also 'ntb', keep 'Ntb'
        if desc.lower() == 'ntb' or cont.lower() == 'ntb':
            return 'Ntb'
        else:
            return f"{desc} {cont}".strip()
    return short

def update_short_description_column(df):
    """Pas de shortdescription-kolom aan met de logica uit _update_short_description_row."""
    df['shortdescription'] = df.apply(_update_short_description_row, axis=1)
    return df

def clean_learningoutcomes_column(df):
    """Schoon de learningoutcomes-kolom op."""
    df['learningoutcomes'] = df['learningoutcomes'].replace(
        {
            r'(?i)^(ntb|n\.t\.b\.|n\.n\.b\.|volgt|nog nader te bepalen|nog te formuleren|nog niet bekend|nog te bepalen|\s*)$':
            'Nader te bepalen'
        },
        regex=True
    )

    df['learningoutcomes'] = df['learningoutcomes'].fillna('Nader te bepalen')
    return df

def parse_tags(value):
    """Zorg dat module_tags altijd een Python-lijst wordt."""
    if isinstance(value, list):
        return value
    if pd.isna(value):
        return []
    if isinstance(value, str):
        s = value.strip()
        if not s:
            return []
        if s.startswith('[') and s.endswith(']'):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, list):
                    return parsed
            except (ValueError, SyntaxError):
                pass
        return [p.strip() for p in s.split(',') if p.strip()]
    return []

def clean_tag_phrase(tag, stopwords=DUTCH_STOPWORDS):
    """
    Maak één tag schoon:
    - lowercasing
    - stopwoorden verwijderen
    - dubbele woorden binnen dezelfde tag verwijderen
    - multi-word tag blijft één string
    """
    if not isinstance(tag, str):
        tag = str(tag)

    tag = tag.lower().strip()
    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", tag)

    cleaned_tokens = []
    seen = set()
    for tok in tokens:
        if tok in stopwords:
            continue
        if tok in seen:
            continue
        seen.add(tok)
        cleaned_tokens.append(tok)

    return " ".join(cleaned_tokens).strip()

def preprocess_text_for_keybert(description, content, stopwords=DUTCH_STOPWORDS):
    """
    Maak een gecleande tekst voor KeyBERT:
    - description + content samenvoegen
    - lowercase
    - niet-alfanumerieke tekens eruit
    - stopwoorden eruit
    """
    desc = str(description) if description is not None else ""
    cont = str(content) if content is not None else ""
    text = f"{desc} {cont}".lower()

    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)
    filtered = [t for t in tokens if t not in stopwords and len(t) > 2]

    return " ".join(filtered)

def generate_tags_from_text(description, content,
                            kw_model=None,
                            stopwords=DUTCH_STOPWORDS,
                            max_tags=10):
    """
    Genereer tags uit description + content.
    - Als kw_model is meegegeven: gebruik KeyBERT (1–2-grams).
    - Anders: simpele token-based fallback.
    """
    # --- met KeyBERT ---
    if kw_model is not None:
        clean_text = preprocess_text_for_keybert(description, content, stopwords)
        if not clean_text.strip():
            return []

        keywords = kw_model.extract_keywords(
            clean_text,
            keyphrase_ngram_range=(1, 2),  # 1- of 2-woord tags
            stop_words=list(stopwords),
            top_n=max_tags
        )
        # keywords = [(phrase, score), ...]
        tags = []
        seen = set()
        for phrase, score in keywords:
            phrase = phrase.strip()
            if not phrase:
                continue
            if phrase in seen:
                continue
            seen.add(phrase)
            tags.append(phrase)
        return tags

    # --- fallback zonder KeyBERT ---
    desc = str(description) if description is not None else ""
    cont = str(content) if content is not None else ""
    text = f"{desc} {cont}".lower()

    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)

    tags = []
    seen = set()
    for tok in tokens:
        if tok in stopwords:
            continue
        if len(tok) <= 2:
            continue
        if tok in seen:
            continue
        seen.add(tok)
        tags.append(tok)
        if len(tags) >= max_tags:
            break

    return tags

def tags_are_empty_or_ntb(tags):
    """True als lijst leeg is of alleen varianten van 'ntb' bevat."""
    if not tags:
        return True
    normalized = [str(t).strip().lower() for t in tags if str(t).strip()]
    if not normalized:
        return True
    return all(t in {'ntb', 'n.t.b', 'nog te bepalen'} for t in normalized)

def process_module_tags_row(row,
                            tag_col='Module_tags',
                            stopwords=DUTCH_STOPWORDS,
                            kw_model=None):
    """
    Schoon één rij op:
    - parse bestaande tags
    - als leeg / ntb -> genereer nieuwe tags uit description + content (KeyBERT)
    - anders: bestaande tags schoonmaken + duplicates verwijderen
    """
    raw_tags = parse_tags(row.get(tag_col, []))

    # Als tags leeg of 'ntb' -> genereren uit description + content
    if tags_are_empty_or_ntb(raw_tags):
        gen_tags = generate_tags_from_text(
            row.get('description', ''),
            row.get('content', ''),
            kw_model=kw_model,
            stopwords=stopwords
        )
        return gen_tags

    # Bestaande tags opschonen
    cleaned_tags = []
    seen_tags = set()

    for tag in raw_tags:
        cleaned = clean_tag_phrase(tag, stopwords=stopwords)
        if not cleaned:
            continue
        if cleaned in seen_tags:
            continue
        seen_tags.add(cleaned)
        cleaned_tags.append(cleaned)

    # Als alles is weggefilterd, alsnog fallback naar description+content
    if not cleaned_tags:
        cleaned_tags = generate_tags_from_text(
            row.get('description', ''),
            row.get('content', ''),
            kw_model=kw_model,
            stopwords=stopwords
        )

    return cleaned_tags

def clean_and_fill_module_tags(df,
                               tag_col='Module_tags',
                               stopwords=DUTCH_STOPWORDS,
                               kw_model=None):
    """
    Past process_module_tags_row toe op de hele DataFrame.
    Gebruikt KeyBERT als kw_model is meegegeven.
    """
    df = df.copy()
    if tag_col not in df.columns:
        return df

    df[tag_col] = df.apply(
        lambda row: process_module_tags_row(
            row,
            tag_col=tag_col,
            stopwords=stopwords,
            kw_model=kw_model
        ),
        axis=1
    )
    return df

import ast


def clean_stopwords_from_existing_tags(test):

    test["module_tags"] = test["module_tags"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    from nltk.corpus import stopwords

    stop_words = set(stopwords.words('dutch'))

    test["module_tags"] = test["module_tags"].apply(
        lambda lst: [w for w in lst if w.lower() not in stop_words]
    )
    return test

def regenerate_tags_when_ntb(df,
                            tag_col="module_tags",
                            desc_col="description",
                            cont_col="content",
                            stopwords=DUTCH_STOPWORDS,
                            kw_model=None):
    """
    - Detects rows where module_tags contains only ['ntb'] or variants.
    - Removes those tags.
    - Generates NEW tags using KeyBERT based on description + content.
    """

    def is_ntb_tag_list(tag_list):
        if not tag_list:
            return True
        cleaned = [str(t).strip().lower() for t in tag_list]
        return all(t in {"ntb", "n.t.b", "n.t.b.", "nog te bepalen"} for t in cleaned)

    def generate_new_tags(row):
        text_desc = row.get(desc_col, "")
        text_cont = row.get(cont_col, "")

        clean_text = preprocess_text_for_keybert(text_desc, text_cont, stopwords)

        if not clean_text.strip():
            return []

        keywords = kw_model.extract_keywords(
            clean_text,
            keyphrase_ngram_range=(1, 2),
            stop_words=list(stopwords),
            top_n=10
        )

        tags = []
        seen = set()
        for phrase, score in keywords:
            p = phrase.strip()
            if p and p not in seen:
                seen.add(p)
                tags.append(p)

        return tags

    # 1. Ensure tags are lists (using your existing parser)
    df[tag_col] = df[tag_col].apply(parse_tags)

    # 2. Regenerate tags only where NTB-tags are detected
    df[tag_col] = df.apply(
        lambda row: generate_new_tags(row) if is_ntb_tag_list(row[tag_col]) else row[tag_col],
        axis=1
    )

    return df

def fill_missing_descriptions(df,
                              desc_col="description",
                              cont_col="content",
                              short_col="shortdescription"):
    """
    Vervangt lege of NTB-achtige description/content door shortdescription.
    """
    
    # Alle varianten die betekenen: "niet bekend", "nog te bepalen", etc.
    NTB_VALUES = {
        "", " ", "ntb", "n.t.b.", "n.t.b", "nnb", "volgt",
        "nader te bepalen", "nog nader te bepalen",
        "nog te bepalen", "nog niet bekend", "none", "nan"
    }

    def is_missing_or_ntb(value):
        if value is None:
            return True
        text = str(value).strip().lower()
        return text in NTB_VALUES

    def fix_row(row):
        short = str(row.get(short_col, "")).strip()
        
        # Alleen vervangen als shortdescription wél bruikbaar is
        if is_missing_or_ntb(short):
            return row  # niets invullen als shortdesc zelf "ntb" is
        
        # Description vervangen?
        if is_missing_or_ntb(row.get(desc_col)):
            row[desc_col] = short
        
        # Content vervangen?
        if is_missing_or_ntb(row.get(cont_col)):
            row[cont_col] = short
        
        return row

    # Pas op elke rij toe
    return df.apply(fix_row, axis=1)

def clean_df(df, kw_model=None):
    """Pipeline: voert alle cleanup-stappen uit op een kopie van de dataframe."""
    df_copy = df.copy()

    df_copy = remove_color_columns(df_copy)
    df_copy = remove_rows_by_id(df_copy, ids_to_remove=[293, 294, 295, 232])
    df_copy = update_short_description_column(df_copy)
    df_copy = clean_learningoutcomes_column(df_copy)
    df_copy = clean_and_fill_module_tags(
        df_copy,
        tag_col='Module_tags',
        stopwords=DUTCH_STOPWORDS,
        kw_model=kw_model
    )
    # NEW: regenerate where only NTB existed
    df_copy = regenerate_tags_when_ntb(df_copy,
                                      tag_col="module_tags",
                                      desc_col="description",
                                      cont_col="content",
                                      stopwords=DUTCH_STOPWORDS,
                                      kw_model=kw_model)
    df_copy = clean_stopwords_from_existing_tags(df_copy)
    df_copy = fill_missing_descriptions(df_copy)

    return df_copy



# 1) maak eerst je KeyBERT-model
kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

# 2) run de cleaning + taggeneratie
df_copy = clean_df(df, kw_model=kw_model)

df_copy.to_csv("Uitgebreide_VKM_dataset_cleaned3.csv", index=False)
display(df_copy)

,id,name,shortdescription,description,content,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,Kennismaking met Psychologie,"Brein, gedragsbeinvloeding, ontwikkelingspsych...",In deze module leer je hoe je gedrag van jezel...,In deze module leer je hoe je gedrag van jezel...,15,Den Bosch,58,NLQF5,A. Je beantwoordt vragen in een meerkeuze kenn...,"[brein, gedragsbeinvloeding, ontwikkelingspsyc...",0.54,319,1,79,2025-12-24
1,160,Learning and working abroad,"Internationaal, persoonlijke ontwikkeling, ver...",Studenten kiezen binnen de (stam) van de oplei...,Studenten kiezen binnen de (stam) van de oplei...,15,Den Bosch,58,NLQF5,De student toont professioneel gedrag conform ...,"[internationaal, persoonlijke, ontwikkeling, v...",0.92,172,5,56,2025-12-20
2,161,Proactieve zorgplanning,"Proactieve zorgplanning, cocreatie, ziekenhuis",Het Jeroen Bosch ziekenhuis wil graag samen me...,Het Jeroen Bosch ziekenhuis wil graag samen me...,15,Den Bosch,59,NLQF5,De student past pro actieve zorgplanning toe b...,"[proactieve, zorgplanning, cocreatie, ziekenhuis]",0.78,217,5,55,2025-09-23
3,162,Rouw en verlies,"Rouw & verlies, palliatieve zorg & redeneren, ...",In deze module wordt stil gestaan bij rouw en ...,In deze module wordt stil gestaan bij rouw en ...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"[rouw, verlies, palliatieve, zorg, redeneren, ...",0.69,454,1,54,2025-10-25
4,163,Acuut complexe zorg,"Acute zorg, complexiteit, ziekenhuis, revalidatie",In deze module kunnen studenten zich verdiepen...,In deze module kunnen studenten zich verdiepen...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"[acute, zorg, complexiteit, ziekenhuis, revali...",0.40,178,5,38,2025-11-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,393,Ethiek & kritisch denken,"ethiek, filosofie, kritisch denken, ethisch ha...",Ethiek en ethisch handelen wordt steeds belang...,Ethiek en ethisch handelen wordt steeds belang...,30,Breda en Den Bosch,124,NLQF6,De student demonstreert persoonlijke groei op ...,"[ethiek, filosofie, kritisch, denken, ethisch,...",0.37,248,5,32,2025-10-17
207,394,Avans Innovative Studio Junior,"persoonlijke ontwikkeling, interdisciplinair, ...",Je leert hoe welke waarde jouw werk heeft. Hoe...,Je leert hoe welke waarde jouw werk heeft. Hoe...,15,Breda,92,NLQF6,De student demonstreert persoonlijke groei op ...,"[persoonlijke, ontwikkeling, interdisciplinair...",0.73,299,5,62,2025-09-01
208,395,Avans Innovative Studio Senior,Ook heb je de mogelijkheid om met creatieve on...,Ook heb je de mogelijkheid om met creatieve on...,Ook heb je de mogelijkheid om met creatieve on...,30,Breda,92,NLQF6,De student demonstreert persoonlijke groei op ...,"[creatieve ondernemers, ondernemers werkveld, ...",0.30,105,5,62,2025-11-29
209,396,Succesvol managen van een project,"Project Evaluatie, Scrum, Planning, Agile, Pri...","Leren om succesvol een project (innovatie, cha...","Leren om succesvol een project (innovatie, cha...",15,Breda,125,NLQF5,"1) Inzicht/kennis in de diverse factoren, zoal...","[project, evaluatie, scrum, planning, agile, p...",0.40,135,5,67,2025-09-14
